In [ ]:
# For passwords & database access:
import psycopg2
import psycopg2.extras
import getpass

# For plotting
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import display
import ipywidgets

# For math
import numpy as np
import pandas as pd
from decam_utils import *
from scipy.optimize import curve_fit
from scipy.stats import linregress

import time

In [ ]:
# Logging in
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")
db = psycopg2.connect( f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'" )

db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
minnumdet = 5 # Minimum number of detections on the first night of data for the filter to consider a candidate
maxmgerr = 0.03 # Maximum median mag error (In each candidate-night-filter)
numsig = 2 # How many sigma is a 'detection'?
magamp = 0.0 # Minimum magnitude change throughout the candidates first night of detection

In [ ]:
# Get a list of every MJD for which we have an image
query = ("SELECT e.mjd FROM exposures e "
         "WHERE (e.proposalid = '2021A-0113' "
         "OR e.proposalid = '2021B-0149') "
         "AND (e.ra < 50 "
         "OR e.dec > 0) "
         "LIMIT 10000000")
cursor.execute( query, )

datesquery = np.array(cursor.fetchall())

In [ ]:
# Make a list of every observing night (by rounding to the nearest MJD)
odatesrd = np.unique( np.round( datesquery ) )
datemsks = [np.where(np.abs(datesquery-i) < 0.5) for i in odatesrd]
odates = np.array([np.mean(datesquery[i]) for i in datemsks])

In [ ]:
def lin(x,m,b):
    return m*x + b

In [ ]:
fr = []
frdates = []
filidx = []
fitpararr = []
amps = []
maxmags = []
medmgerrs = []
slopeerrs = []
numdets = []

t1 = time.time() # should be about 30 seconds

goodfnms = good_fnms(cursor)[0] # Pulls all the exposure IDs of science images 

for field in ["COSMOS", "ELIAS"]:
    for fil in ["g","r","i"]:
        tempfr = [] # List for the names of fast-rising candidates
        tempfrdates = [] # List for the dates on which fast riser behavior was observed
        
        oldcands = np.array( [] ) # used to check if candidates have been detected before

        for i in range(len(odatesrd)):
            # Loop through each observing night
            # Grab all detections within a day of the night in question
            lodate = odatesrd[i] - 0.5
            hidate = odatesrd[i] + 0.5
            if field == "COSMOS":
                query = ("SELECT o.candidate_id, o.id, e.mjd, o.mag, rbs.rb, e.id, o.magerr FROM objects o "
                         "JOIN subtractions s ON s.id=o.subtraction_id "
                         "JOIN exposures e on s.exposure_id = e.id "
                         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
                         "WHERE q3c_radial_query(e.ra,e.dec,150,2.2,3) "
                         "AND rbs.rb > 0.4 "
                         "AND e.mjd > %s "
                         "AND e.mjd < %s "
                         "AND e.filter = %s "
                         "AND (e.proposalid = '2021A-0113' "
                         "OR e.proposalid = '2021B-0149') "
                         "LIMIT 10000000")
            elif field == "ELIAS":
                query = ("SELECT o.candidate_id, o.id, e.mjd, o.mag, rbs.rb, e.id, o.magerr FROM objects o "
                         "JOIN subtractions s ON s.id=o.subtraction_id "
                         "JOIN exposures e on s.exposure_id = e.id "
                         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
                         "WHERE q3c_radial_query(o.ra,o.dec,8.5,-43.5,2) "
                         "AND rbs.rb > 0.4 "
                         "AND e.mjd > %s "
                         "AND e.mjd < %s "
                         "AND e.filter = %s "
                         "AND (e.proposalid = '2021A-0113' "
                         "OR e.proposalid = '2021B-0149') "
                         "LIMIT 10000000")
            cursor.execute( query, ( lodate, hidate, fil, ) )

            # Store that nights detections in an array
            tempres = np.array( cursor.fetchall(), dtype=str ).transpose()

            if len( tempres ) > 0: # Takes care of errors from gaps in the observations by skipping empty nights
                # Cut out duplicate objects
                dupearr = np.array( [tempres[0], tempres[2], tempres[3], tempres[4], tempres[5], tempres[6]] ).transpose()
                dupearr = [ " ".join(j) for j in dupearr ]
                dupearr, ind = np.unique( dupearr, return_index=True )

                # Rejoin the array without the duplicates
                uarr = np.array( [ i.split(" ") for i in dupearr ] ).transpose()
                res = np.array( [uarr[0], tempres[1][ind], uarr[1], uarr[2], uarr[3], uarr[4], uarr[5]] )
                
                # Cut out non-science images
                msk = np.isin(res[5], goodfnms)
                res = np.array([i[msk] for i in res])
                
                # Create an array of each unique candidate detected that night
                ucands, uind, ucounts = np.unique( res[0], return_counts=True, return_index=True )
                
                # Isolate those with at least numdet detections
                goodcands = ucands[ ucounts >= minnumdet ]


                # Make an array of detections for each of those candidates, store all of those arrays in "arr"
                arr = np.ones( len( goodcands ), dtype=object )
                for j in range( len( goodcands ) ):
                    arr[j] = res[:,res[0,:] == goodcands[j]]
                
                # Mask out those that 
                # A) Don't change by at least magamp mags (currently minimum is 0)
                # B) Have a median mag error of > 0.03
                # C) Don't have a rise with at least 3-sigma confidence
                good = np.ones( len( arr ), dtype=bool )
                for j in range( len( arr ) ):
                    # A)
                    amp = np.max( arr[j][3].astype(float) ) - np.min( arr[j][3].astype(float) )
                    if amp < magamp:
                        good[j] = False
                    else:
                        ntmjds  = arr[j][2].astype(float)
                        ntmags  = arr[j][3].astype(float)
                        ntmgerr = arr[j][6].astype(float)
                        msk1 = ~np.isnan(ntmags)
                        msk2 = ~np.isnan(ntmgerr)
                        msk = msk1 & msk2
                        # print(np.unique([msk,msk1]))
                        ntmjds, ntmags, ntmgerr  = ntmjds[msk], ntmags[msk], ntmgerr[msk]
                        
                        maxmag = np.min(ntmags)
                        medmgerr = np.median(ntmgerr)
                        
                        # B
                        if medmgerr > maxmgerr:
                            good[j] = False
                        
                        elif arr[j][0][0] in oldcands:
                            good[j] = False
                        # C
                        else:
                            # Using scipy curve_fit
                            # fitpars = curve_fit(lin, ntmjds, ntmags, sigma=ntmgerr )
                            # slope, intercept, slopeerr = fitpars[0][0], fitpars[0][1], fitpars[1][0][0]
                            # if slope + np.abs(slopeerr) > 0:
                            #     good[j]=False

                            # Using np.polyfit, weights, and error in params:
                            fitpars = np.polyfit(ntmjds, ntmags, 1, w=1/ntmgerr, cov=True)
                            slope, intercept = fitpars[0]
                            slopeerr = np.sqrt(fitpars[-1][0][0])
                            numdet = len(ntmjds)
                            if slope + numsig * np.abs(slopeerr) > 0:
                                good[j] = False
                            if good[j] == False:
                                pass

                            # Using linregress and Pearson:
                            # fitpars = linregress(ntmjds, ntmags)
                            # slope, intercept, pearson = fitpars[0], fitpars[1], fitpars[2]
                            # if (pearson > -0.5) | (np.isnan(pearson)):
                            #     good[j] = False

                            else:
                                # Save the parameters of the good ones
                                fitpararr.append([slope,intercept])
                                amps.append(amp)
                                maxmags.append(maxmag)
                                medmgerrs.append(medmgerr)
                                slopeerrs.append(slopeerr)
                                numdets.append(numdet)
                    
                # Apply those cuts and append the candidates that passed to fr and frdates
                for j in range( len( arr[good] ) ):
                    tempfr.append( arr[good][j][0][0] )
                    tempfrdates.append( [lodate, hidate] )
            
            # Make a note of all candidates detected this night (for cutting out things that have been previously detected)
            try:
                oldcands = np.append( oldcands, ucands )
            except NameError:
                pass
            fr.append(tempfr)
            frdates.append(tempfrdates)
        filidx.append([len(j) for j in fr][-1])
        
x = []
y = []
ind = np.unique(fr, return_index=True)[1]
for i in [fr[ind] for ind in sorted(ind)]:
    x.extend(i)
for i in [frdates[ind] for ind in sorted(ind)]:
    y.extend(i)
fr = x
frdates = y

fitpararr = np.array(fitpararr)
amps = np.array(amps)
maxmags = np.array(maxmags)

t2 = time.time()
print(t2-t1)

del x, y, ind

In [ ]:
### Checks whether each passed candidate's night of fast rise was on it's first or only night

fn = []
on = []
for i in range(len(fr)):
    query = ("SELECT o.mag, e.mjd FROM objects o "
             "JOIN subtractions s ON s.id=o.subtraction_id "
             "JOIN exposures e on s.exposure_id = e.id "
             "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
             "WHERE rbs.rb > 0.4 "
             "AND (e.proposalid = '2021A-0113' "
             "OR e.proposalid = '2021B-0149') "
             "AND o.candidate_id = %s "
             "ORDER BY e.mjd "
             "LIMIT 10000000")
    cursor.execute( query, ( fr[i], ) )
    tempres = np.array( cursor.fetchall(), dtype=float ).transpose()
    if len(tempres) == 0:
        fn.append(False)
        on.append(False)
    else:
        lastnight = float(tempres[1][-1])
        firstnight = float(tempres[1][0])
        if firstnight < frdates[i][0]:
            fn.append(False)
            on.append(False)
        else:
            fn.append(True)
            if lastnight > frdates[i][1]:
                on.append(False)
            else:
                on.append(True)

In [ ]:
len(fr), len(fitpararr)

In [ ]:
### Combines all of the information collected above into one big dataframe
### Each candidate gets at least one row (cands that pass in multiple filters get more rows)

df = pd.DataFrame(index = range(len(fr)), columns = ["CandID", "Field", "Filter", "Night", "DelMag", "PeakMag", "FirstNight", "OnlyNight", "Slope", "Intercept", "MedErr"])

for i in range(len(fr)):
    df["CandID"][i] = fr[i]
    if i < sum(filidx[:3]):
        df["Field"][i] = "COSMOS"
    else: 
        df["Field"][i] = "ELAIS"
    df["Night"][i] = frdates[i][0]+0.5
    df["Slope"][i] = fitpararr[i][0]
    df["Intercept"][i] = fitpararr[i][1]
    df["DelMag"][i] = amps[i]
    df["PeakMag"][i] = maxmags[i]
    df["MedErr"][i] = medmgerrs[i]
    df["FirstNight"][i] = fn[i]
    df["OnlyNight"][i] = on[i]
    if i < filidx[0]:
        df["Filter"][i] = "g"
    elif (i >= filidx[0]) & (i < sum(filidx[:2])):
        df["Filter"][i] = "r"
    elif (i >= sum(filidx[:2])) & (i < sum(filidx[:3])):
        df["Filter"][i] = "i"
    elif (i >= sum(filidx[:3])) & (i < sum(filidx[:4])):
        df["Filter"][i] = "g"
    elif (i >= sum(filidx[:4])) & (i < sum(filidx[:5])):
        df["Filter"][i] = "r"
    elif (i >= sum(filidx[:5])): #& (i < sum(filidx[:6])):
        df["Filter"][i] = "i"
### This takes care of duplicates that arise from having observations closer together than usual
df = df.drop_duplicates(subset=["CandID", "Field", "Filter", "DelMag", "PeakMag"])
try:
    df = df.reset_index(drop=True)
except ValueError:
    pass
fr = df["CandID"]
frdates = [[df["Night"][i]-0.5,df["Night"][i]+0.5] for i in range(len(fr))]

In [ ]:
### Summary of Results:
print("There were %s total first-night fast risers" % len(df))
print("Spread over %s candidates (repeats mean detection in more than one filter)" % len(np.unique(df["CandID"])))
print("%s in g, %s in r, and %s in i" % (len(df[df["Filter"]=="g"]), len(df[df["Filter"]=="r"]), len(df[df["Filter"]=="i"])))
print("%s in COSMOS, %s in ELAIS" % (len(df[df["Field"]=="COSMOS"]), len(df[df["Field"]=="ELAIS"])))
# print("%s of these rose quickly on their first night" % len(df[df["FirstNight"]==True]))
print("%s of those were only detected on one night" % len(df[df["OnlyNight"]==True]))

#### Overlap with "probably-real" cands
This section checks which of the candidates that passed the first-night fast rise filter were also passed by the "probably-real" cuts described in the paper

In [ ]:
bigarr = np.loadtxt('../candidate_nightly_epochs_files/candidate_lightcurves.dat', dtype=str)
prob_real_cands = np.unique(bigarr.transpose()[1]) # list of all "probably-real" candidates
frcands = df["CandID"].values
# frcands = np.unique(df["CandID"].values) # List of all cands that passed this notebook's filter
# frcands = np.unique(df["CandID"][df["OnlyNight"]==True].values) # List of all single-night cands that passed this notebook's filter
# frcands = np.unique(df["CandID"][df["OnlyNight"]==False].values) # List of all multi-night cands that passed this notebook's filter

In [ ]:
prob_real_msk = np.isin(frcands,prob_real_cands)
prob_not_real_frcands = frcands[~prob_real_msk]
prob_real_frcands = frcands[prob_real_msk]
df["LikelyReal"] = prob_real_msk

In [ ]:
prob_real_frcands

In [ ]:
# Why did those that failed fail?
for i in prob_not_real_frcands:
    query = ("SELECT o.candidate_id, o.id, rbs.rb FROM objects o "
             "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
             "WHERE o.candidate_id = %s ")
    cursor.execute(query, (i,))
    arr = np.array(cursor.fetchall()).transpose()
    print(i)
    print("Number of detections: {}".format(len(np.unique(arr[2])))) # this *should* take care of dupe objs, as it's very unlikely that two objs would have same rb
    print("Mean RB score: {}".format(np.mean(arr[2].astype(float))))

#### First-night likely-real fast riser plot

In [ ]:
filts = ["g", "r", "i"]
all_fshapes = ['o','s','*']
all_fsizes  = [1, 1, 3]
all_fcolors = ["green","red","blue"]

fig, ax = plt.subplots(7,2,figsize=(15,22),gridspec_kw={'width_ratios': [3, 1]})
# ax[0,1].errorbar(0,0,0,fmt=all_fshapes[0], ms=all_fsizes[0]+3, alpha=1, mew=0, color="green", label="g") # for legend
lylimsdict = {"DC21bae": [19.35, 20.65],#
            "DC21kkqh": [19.6,21],
            "DC21kkte": [20.4,23],#
            "DC21kldj": [16,18],
            "DC21knzx": [20,21],#
            "DC21koer": [20,23],
            "DC21kvqx": [20.2,21.5],
            "DC21krys": [20.5,21]}
rylimsdict = {"DC21bae": [19.8, 20.2],
            "DC21kkqh": [19.5,20.1],
            "DC21kkte": [20.7,21],
            "DC21kldj": [17,17.2],
            "DC21knzx": [20.2,20.5],
            "DC21koer": [20.7,21.25],
            "DC21kvqx": [20.2,20.75],
            "DC21kluc": [20.75,21.7],
            "DC21kptk": [19.5,21],
            "DC21krys": [20.5,21.2]}
for j, candid in enumerate(np.unique(df["CandID"][(df["LikelyReal"]==True) & (df["OnlyNight"]==False)])):
    i = np.where(df["CandID"].values==candid)[0][0]
    # if df["Field"][i] == "COSMOS":
    #     all_fcolors = ["darkgreen","red","brown"]
    # elif df["Field"][i] == "ELAIS": 
    #     all_fcolors = ["limegreen","darkorange","peru"]
    for f in range(len(filts)):
        query = ('SELECT c.id, e.mjd, o.mag, o.magerr, o.ra, o.dec FROM objects o '
                 'JOIN candidates c ON c.id=o.candidate_id '
                 'JOIN subtractions s ON s.id=o.subtraction_id '
                 'JOIN exposures e ON e.id=s.exposure_id '
                 'JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 '
                 'WHERE c.id=%s '
                 'AND e.filter=%s '
                 'AND rbs.rb > 0.4')
        cursor.execute( query, ( candid, filts[f], ) )
        try:
            ax[j,0].set_ylim(lylimsdict[candid])
        except:
            pass
        array = np.array( cursor.fetchall() ).transpose()
        array = rm_dupes(array)
        
        # Grab data for full lightcurve
        array2 = plotlc(candid, cursor, show_plot=False)

        ### At left, plot full lightcurve
        if len(array2) != 0:
            # ax[j,0].errorbar( array2[0][f], array2[1][f], yerr=array2[2][f], fmt=all_fshapes[f], ms=all_fsizes[f], alpha=0.8, mew=0, color=all_fcolors[f])
            ax[j,0].scatter( array2[0][f], array2[1][f], marker=all_fshapes[f], lw=all_fsizes[f], alpha=0.8, color=all_fcolors[f])
            msk = ~np.isin(odatesrd, np.round(array2[0][f]))
            
        ### at right, zoom in on one epoch
        if len(array) > 0:
            omjds  = array[1].astype(float)
            omags  = array[2].astype(float)
            omerrs = array[3].astype(float)
            ax[j,1].errorbar(omjds, omags, omerrs, fmt=all_fshapes[f], ms=all_fsizes[f]+4, alpha=1, mew=0, color=all_fcolors[f], label=filts[f])
            ax[j,0].tick_params(labelbottom=True)
            ax[j,1].tick_params(labelbottom=True)
            
            if df["Filter"][i] == filts[f]:
                # (omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)
                mjd = np.median(omjds[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
                mag = np.median(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
                minmag = np.min(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])
                maxmag = np.max(omags[(omjds>df["Night"][i]-0.5) & (omjds<df["Night"][i]+0.5)])

                ax[j,1].set_xlim(mjd-0.07, mjd+0.07)
                try:
                    ax[j,1].set_ylim(rylimsdict[candid])
                except:
                    pass
                # frmsk = np.where((array2[0][f] > mjd-0.5) & (array2[0][f] < mjd+0.5))[0][0]
                # ax[j,0].errorbar( array2[0][f][frmsk], array2[1][f][frmsk], yerr=array2[2][f][frmsk], fmt=all_fshapes[f], ms=all_fsizes[f], alpha=0.8, mew=0, color="black" )
                
                xarr = np.linspace(mjd-1, mjd+1, 10)
                ax[j,1].plot(xarr, lin(xarr, df["Slope"][i], df["Intercept"][i]),c=all_fcolors[f], alpha=0.2)
                # ax[j,0].plot(xarr, lin(xarr, df["Slope"][i], df["Intercept"][i]),c=all_fcolors[f], alpha=0.2)
            if (candid == "DC21koer") & (f==0):
                k=df[df["CandID"]==candid].index[1]
                ax[j,1].plot(xarr, lin(xarr, df["Slope"][k], df["Intercept"][k]),c="red", alpha=0.2)
    ax[j,1].invert_yaxis()
    ax[j,0].invert_yaxis()
    if candid=="DC21krys":
        ax[j,0].text(.95,.05,candid,fontsize=16,transform=ax[j,0].transAxes,horizontalalignment="right",verticalalignment="bottom")
    else:
        ax[j,0].text(.95,.95,candid,fontsize=16,transform=ax[j,0].transAxes,horizontalalignment="right",verticalalignment="top")
    ax[j,0].set_ylabel("Magnitude", fontsize=18)
    ax[j,0].tick_params(axis='both', which='major', labelsize=12)
    ax[j,1].tick_params(axis='both', which='minor', labelsize=12)

fig.supxlabel('MJD', fontsize=18)
ax[0,1].legend(loc="upper left", frameon=True, fontsize=14)
ax[0,0].set_title("Full lightcurve", fontsize=18)
ax[0,1].set_title("First night detections", fontsize=18)
fig.tight_layout()
plt.savefig("LRfastrisers")

#### Non-asteroid onlynight cands menagerie plot

In [ ]:
### Colors to use for COSMOS-g, -r, -i and ELAIS-g, -r, -i
clr = ['darkgreen', 'firebrick', 'saddlebrown', 'limegreen', 'orange', 'lightcoral']

all_filters = ['g','r','i']
all_fcolors = ['green','red','blue']
all_fshapes = ['o','s','*']
all_fsizes  = [5, 5, 8]
ncols = 3

tx = np.array(df[(df["Field"]=="ELAIS") & (df["OnlyNight"]==True)].index)
# tx = np.array(df[df["OnlyNight"]==True].index)

print(len(tx))
fig, ax = plt.subplots( int(np.ceil(len(tx) / ncols)), ncols, figsize=(16,(int(np.ceil(len(tx) / ncols)))/5*16), sharex=False, sharey=False )

t = 0
for i in range(int(np.ceil(len(tx) / ncols))):
    ax[i,0].set_ylabel('Magnitude', fontsize=14)
    for j in range(ncols):
        if t < len(tx):
            cand = df["CandID"][tx].values[t]
            if df["Field"][tx[t]] == "COSMOS":
                all_fcolors = ["darkgreen","red","brown"]
            elif df["Field"][tx[t]] == "ELAIS": 
                all_fcolors = ["limegreen","darkorange","peru"]
            for f in all_filters:
                # f = df["Filter"][tx[t]]
                query = ("SELECT e.mjd, o.mag, o.magerr FROM objects o "
                         "JOIN subtractions s ON s.id=o.subtraction_id "
                         "JOIN exposures e ON e.id=s.exposure_id "
                         "JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 "
                         "WHERE o.candidate_id = %s "
                         "AND rbs.rb>0.4 "
                         "AND e.filter=%s")
                cursor.execute(query,(cand,f,))
                data = np.array(cursor.fetchall()).transpose()
                try:
                    cmjds, cmags, cmagerrs = data[0],data[1],data[2]
                    if f=='g':
                        f = 0
                    elif f=='r':
                        f=1
                    elif f=="i":
                        f=2
                    
                    ax[i,j].errorbar(cmjds, cmags, yerr=cmagerrs, marker=all_fshapes[f],lw=0,elinewidth=1, mew=0,ms=all_fsizes[f], color=all_fcolors[f], label=all_filters[f])
                    
                    ylims = np.array(ax[i,j].dataLim)
                    if all_filters[f]==df["Filter"][tx[t]]:
                        start = np.min(cmjds)
                        finish = np.max(cmjds)
                        xarr = np.linspace(start-0.03, finish+0.03, 100)
                        ax[i,j].plot(xarr,lin(xarr,df["Slope"][tx[t]],df["Intercept"][tx[t]]),color=all_fcolors[f], alpha=0.5)
                    
                    ax[i,j].text(0.9, 0.1, cand, fontsize=14, color='black', 
                                 verticalalignment='bottom', horizontalalignment='right',  transform=ax[i,j].transAxes)
                except IndexError:
                    pass
            ax[i,j].invert_yaxis()
            t += 1
        else:
            fig.delaxes(ax[i,j])
            for k in range(j):
                ax[i,k].set_xlabel('MJD', fontsize=14)
            for k in range(ncols-j):
                ax[i-1,k+j].set_xlabel('MJD', fontsize=14)
                

plt.tight_layout()
plt.savefig("fastriser_menagerie")
plt.show()

In [ ]:
for i in df["CandID"][(df["OnlyNight"]==True) & (df["Field"]=="ELAIS")].values:
    query = ( 'SELECT cand.id, e.mjd,e.filename,e.filter,c.sci_jpeg,c.ref_jpeg,c.diff_jpeg FROM cutouts c '
              'INNER JOIN objects o ON c.object_id=o.id '
              'INNER JOIN candidates cand ON o.candidate_id=cand.id '
              'INNER JOIN subtractions s ON o.subtraction_id=s.id '
              'INNER JOIN exposures e ON s.exposure_id=e.id '
              'JOIN objectrbs as rbs ON o.id=rbs.object_id AND rbs.rbtype_id=2 '
              'WHERE cand.id=%s '
              'AND rbs.rb > 0.4 '
              'ORDER BY e.mjd '
              'LIMIT 1')
    cursor.execute( query, ( i, ) )
    for row in cursor:
        label = ipywidgets.HTML( f'<b>Candidate</b>{row["id"]}<br><b>{row["filename"]}</b><br><b>Filter:</b> {row["filter"]}' )
        newimg = ipywidgets.Image( value=bytes(row["sci_jpeg"]), width=102, height=102, format='jpeg' )
        refimg = ipywidgets.Image( value=bytes(row["ref_jpeg"]), width=102, height=102, format='jpeg' )
        subimg = ipywidgets.Image( value=bytes(row["diff_jpeg"]), width=102, height=102, format='jpeg' )
        box = ipywidgets.HBox( [ newimg, refimg, subimg, label ] )
        display( box )
    print("")

#### MP checker
This section generates dates and coords in the formats needed for the MPchecker (https://minorplanetcenter.net/cgi-bin/checkmp.cgi)

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

In [ ]:
for x, i in enumerate(df["CandID"][df["OnlyNight"]==True]):
    query = ("SELECT e.mjd, o.ra, o.dec FROM objects o "
             "JOIN subtractions s ON s.id=o.subtraction_id "
             "JOIN exposures e ON e.id=s.exposure_id "
             "WHERE o.candidate_id = %s "
             "LIMIT 1 ")
    cursor.execute(query,(i,))
    radec = cursor.fetchall()[0]
    datetime = Time(radec[0], format="mjd").iso
    print(i)
    frac = str(round(sum([float(datetime[11:13]), float(datetime[14:16])/60]) / 24, 2))[1:]
    print(datetime[:10]+frac )
    coords = SkyCoord(radec[1], radec[2], unit='deg').to_string('hmsdms')
    print(coords[:2],coords[3:5], coords[6:11])
    print(coords[19:22],coords[23:25], coords[26:31])
    print("")